# Modelo prectivos de precios de inmuebles

Para esta sesión trabajaremos con una base de datos sobre los precios de inmuebles en la ciudad de Ames, Iowa. La base se compone de 2930 registros y contiene un gran número de atributos.

Nuestro objetivo es generar un modelo que prediga de forma adecuada los precios de inmuebles, medidos con la variable `S​ale_Price​`.

## Preparación del ambiente de trabajo

- Se importan las librerías clásicas a utilizar.
- Para este ejercicio implementaremos árboles de regresión, por lo que deberá importar la clase ​DecisionTreeRegressor​.
- De manera adicional importe las funciones y clases necesarias para generar un desempeño de métricas en problemas de regresión, división de muestras y búsqueda de grilla con validación cruzada.
- Se eliminan la columna 'Unnamed: 0' cuando cargue los datos.

In [24]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import pickle

df = pd.read_csv('ames_housing.csv').drop(columns='Unnamed: 0')

## Feature engineering

- Se identifica si el `​dtype` de cada `​pd.Serie` en nuestra base de datos se considera `'object'`​ o no. Para todas las variables que sean `​'object'`​, realice lo siguiente:
    + Se generan una recodificación $k − 1$ en cada variable. Para efectos prácticos sólo necesitan eliminar una de las categorías, no se concentren en especificar la categoría a eliminar. Pueden utilizar la función con la opción `​drop_first` para ello.
    + Utilizando el método `​pd.concat`​, concatene a los atributos creados en la base de datos.
  
> **Tip:** No se olvide de eliminar los atributos recodificados, de esta forma evitará un aumento artificial del desempeño del modelo.


In [25]:
df_objeto = df.select_dtypes(include='object')
df_codificado = pd.get_dummies(df_objeto, drop_first=True)
df = pd.concat([df, df_codificado], axis=1)
df = df.drop(columns=df_objeto.columns)

## Primer modelo

- Se generan muestras de entrenamiento y validación con `​'Sale_Price'` como vector objetivo y los atributos de la base de datos como matriz.
- Recuerde definir el porcentaje de casos en la muestra de validación y una semilla pseudoaleatoria.
- Posteriormente, entrene un árbol de regresión en la muestra de entrenamiento sin modificar los hiper parámetros. Reporte las principales métricas de desempeño.

In [19]:
X = df.drop(columns='Sale_Price')
y = df['Sale_Price']

X_entrenamiento, X_validacion, y_entrenamiento, y_validacion = train_test_split(X, y, test_size=0.2, random_state=42)

arbol_regresion = DecisionTreeRegressor(random_state=42)
arbol_regresion.fit(X_entrenamiento, y_entrenamiento)

y_entrenamiento_pred = arbol_regresion.predict(X_entrenamiento)
y_validacion_pred = arbol_regresion.predict(X_validacion)


rmse_entrenamiento = mean_squared_error(y_entrenamiento, y_entrenamiento_pred, squared=False)
r2_entrenamiento = r2_score(y_entrenamiento, y_entrenamiento_pred)
rmse_validacion = mean_squared_error(y_validacion, y_validacion_pred, squared=False)
r2_validacion = r2_score(y_validacion, y_validacion_pred)


print("RMSE validación:", rmse_validacion)
print("R^2 validación:", r2_validacion)

RMSE entrenamiento: 0.0
R^2 entrenamiento: 1.0
RMSE validación: 46277.64255922713
R^2 validación: 0.7328833928658394


## Importancia relativa

- Se implementa el método `​plot_importance` utilizado en la lectura para reportar la importancia relativa de los atributos.
- Comente sobre cuáles son los principales 10 atributos que afectan la predicción de `Sale_Price​`.
- Se separan estos 10 atributos en una nueva base de datos, junto con el vector objetivo.

In [26]:
importancia = pd.Series(arbol_regresion.feature_importances_, index=X.columns).sort_values(ascending=False)
top_10_caracteristicas = importancia.head(10)
print("Top 10 características:")
print(top_10_caracteristicas)
df_top_10 = pd.concat([X[top_10_caracteristicas.index], y], axis=1)

Top 10 características:
Garage_Cars           0.368422
Gr_Liv_Area           0.200890
Total_Bsmt_SF         0.089019
Exter_Qual_Typical    0.063321
First_Flr_SF          0.041396
Year_Built            0.026110
Second_Flr_SF         0.022622
Longitude             0.016146
Wood_Deck_SF          0.013967
Lot_Area              0.011798
dtype: float64


## Refactorización del modelo y `picklingpd.get_dummies`

- En función de los atributos seleccionados en el ejercicio anterior, vuelva a generar conjuntos de entrenamiento y validación.
- **Dentro de los datos de entrenamiento** genere una búsqueda de grilla con `​GridSearchCV` utilizando los siguientes hiper parámetros:
    + **Máximo de atributos**:​ Evalúe todos los posibles atributos.
    + **Máximo de profundidad**:​ Entre 1 a 32.
    + **Validaciones cruzadas**:​ 5.
- Reporte la mejor combinación de hiper parámetros y su desempeño asociado. Compare el desempeño en la muestra de validación con el modelo por defecto. 

In [28]:
X_top_10 = df_top_10.drop(columns='Sale_Price')
y_top_10 = df_top_10['Sale_Price']
X_train_top_10, X_val_top_10, y_train_top_10, y_val_top_10 = train_test_split(X_top_10, y_top_10, test_size=0.2, random_state=42)
param_grid = {
    'max_features': list(range(1, len(X_train_top_10.columns)+1)),
    'max_depth': list(range(1, 33))
}
grid_search = GridSearchCV(arbol_regresion, param_grid, cv=5)
grid_search.fit(X_train_top_10, y_train_top_10)
mejores_parametros = grid_search.best_params_
mejor_puntaje = grid_search.best_score_
val_rmse_top_10 = mean_squared_error(y_val_top_10, grid_search.predict(X_val_top_10), squared=False)
val_r2_top_10 = r2_score(y_val_top_10, grid_search.predict(X_val_top_10))

print("Mejores parámetros:", mejores_parametros)
print("Mejor puntaje:", mejor_puntaje)
print("RMSE validación (top 10 atributos):", val_rmse_top_10)
print("R^2 validación (top10 atributos):", val_r2_top_10)

Mejores parámetros: {'max_depth': 9, 'max_features': 5}
Mejor puntaje: 0.7739433918795686
RMSE validación (top 10 atributos): 43973.20655787965
R^2 validación (top10 atributos): 0.7588236563541169


### Pickling

- Ahora generamos una serialización de nuestro modelo depurado, y nuestros conjuntos de entrenamiento y validación depurados. Para ello importe el módulo `​pickle​`.
- `pickle` contiene la función dump, que permite guardar el modelo desarrollado. La forma canónica para desarrollar el pickling es:

```python
import pickle
pickle.dump(<OBJETO_CON_EL_MODELO>, open('06-22_nombre-apellido-.sav','wb'))
```


In [30]:
pickle.dump((grid_search, X_train_top_10, X_val_top_10, y_train_top_10, y_val_top_10), open('conjuntos.sav', 'wb'))

print("Archivos guardados correctamente.")

Archivos guardados correctamente.
